# Signal Smoothing by Convolution

This notebook demonstrates how convolution-based filters can smooth noisy signals. We'll explore:

- **Moving Average Filter**: Simple and efficient O(n) smoothing
- **Gaussian Filter**: Smooth filtering with controlled frequency response
- **Calculus Connections**: How discrete operations approximate continuous calculus concepts

## Mathematical Foundation

### Convolution as an Integral

Continuous convolution:
$$(f * g)(t) = \int_{-\infty}^{\infty} f(\tau)g(t-\tau) d\tau$$

Discrete approximation:
$$(f * g)[n] = \sum_{k=-\infty}^{\infty} f[k]g[n-k]$$

### Smoothing Reduces Derivatives

Smoothing filters attenuate high-frequency components, which correspond to rapid changes (high derivatives). The smoothed signal has smaller second derivatives, making it "smoother" in the calculus sense.

In [ ]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

from src.smoothing import moving_average, gaussian_smooth, gaussian_kernel

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed for reproducibility
np.random.seed(42)

print('Setup complete!')

In [ ]:
# Generate synthetic noisy signal
n_points = 2000
frequency = 5.0
noise_std = 0.6

# Time array (2 seconds)
t = np.linspace(0, 2, n_points)

# Clean sinusoidal signal
clean = np.sin(2 * np.pi * frequency * t)

# Add Gaussian noise
noisy = clean + np.random.normal(0, noise_std, n_points)

# Plot original signals
plt.figure(figsize=(14, 5))
plt.plot(t, clean, 'g-', label='Clean Signal', linewidth=2, alpha=0.7)
plt.plot(t, noisy, 'gray', label='Noisy Signal', linewidth=0.5, alpha=0.5)
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.title('Original Signal: Clean vs Noisy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'Generated signal with {n_points} points')
print(f'Signal-to-noise ratio: {np.std(clean)/noise_std:.2f}')

## Moving Average Filter

The moving average filter replaces each point with the average of surrounding points. Our implementation uses the cumulative sum trick for O(n) time complexity.

**Algorithm**: For window size $w$:
$$y[n] = \frac{1}{w} \sum_{k=-(w-1)/2}^{(w-1)/2} x[n+k]$$

In [ ]:
# Apply moving average with different window sizes
ma_5 = moving_average(noisy, window_size=5)
ma_11 = moving_average(noisy, window_size=11)
ma_31 = moving_average(noisy, window_size=31)

# Plot comparison
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(t, noisy, 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
plt.plot(t, ma_5, 'b-', label='Window=5', linewidth=2)
plt.plot(t, ma_11, 'r-', label='Window=11', linewidth=2)
plt.plot(t, ma_31, 'g-', label='Window=31', linewidth=2)
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.title('Moving Average with Different Window Sizes')
plt.legend()
plt.grid(True, alpha=0.3)

# Zoom in on a section
plt.subplot(2, 1, 2)
zoom_start, zoom_end = 500, 700
plt.plot(t[zoom_start:zoom_end], noisy[zoom_start:zoom_end], 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
plt.plot(t[zoom_start:zoom_end], clean[zoom_start:zoom_end], 'k--', label='Clean', linewidth=1.5, alpha=0.7)
plt.plot(t[zoom_start:zoom_end], ma_5[zoom_start:zoom_end], 'b-', label='Window=5', linewidth=2)
plt.plot(t[zoom_start:zoom_end], ma_11[zoom_start:zoom_end], 'r-', label='Window=11', linewidth=2)
plt.plot(t[zoom_start:zoom_end], ma_31[zoom_start:zoom_end], 'g-', label='Window=31', linewidth=2)
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.title('Zoomed View: Effect of Window Size')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Larger windows produce smoother results but introduce more lag')

## Gaussian Filter

The Gaussian filter uses a Gaussian-shaped kernel for weighted averaging. It provides smooth, continuous filtering with well-defined frequency response.

**Kernel**: $G(x) = \exp\left(-\frac{x^2}{2\sigma^2}\right)$

The kernel is normalized so $\sum G(x) = 1$.

In [ ]:
# Visualize Gaussian kernels
sigmas = [1.0, 2.5, 5.0]
plt.figure(figsize=(12, 4))

for sigma in sigmas:
    kernel = gaussian_kernel(sigma)
    x = np.arange(len(kernel)) - len(kernel)//2
    plt.plot(x, kernel, 'o-', label=f'σ={sigma}', linewidth=2, markersize=4)

plt.xlabel('Position')
plt.ylabel('Weight')
plt.title('Gaussian Kernels with Different σ Values')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('Larger σ produces wider, smoother kernels')

In [ ]:
# Apply Gaussian smoothing with different sigmas
gauss_1 = gaussian_smooth(noisy, sigma=1.0)
gauss_2_5 = gaussian_smooth(noisy, sigma=2.5)
gauss_5 = gaussian_smooth(noisy, sigma=5.0)

# Plot comparison
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(t, noisy, 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
plt.plot(t, gauss_1, 'b-', label='σ=1.0', linewidth=2)
plt.plot(t, gauss_2_5, 'r-', label='σ=2.5', linewidth=2)
plt.plot(t, gauss_5, 'g-', label='σ=5.0', linewidth=2)
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.title('Gaussian Smoothing with Different σ Values')
plt.legend()
plt.grid(True, alpha=0.3)

# Zoom in
plt.subplot(2, 1, 2)
plt.plot(t[zoom_start:zoom_end], noisy[zoom_start:zoom_end], 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
plt.plot(t[zoom_start:zoom_end], clean[zoom_start:zoom_end], 'k--', label='Clean', linewidth=1.5, alpha=0.7)
plt.plot(t[zoom_start:zoom_end], gauss_1[zoom_start:zoom_end], 'b-', label='σ=1.0', linewidth=2)
plt.plot(t[zoom_start:zoom_end], gauss_2_5[zoom_start:zoom_end], 'r-', label='σ=2.5', linewidth=2)
plt.plot(t[zoom_start:zoom_end], gauss_5[zoom_start:zoom_end], 'g-', label='σ=5.0', linewidth=2)
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.title('Zoomed View: Effect of σ')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Larger σ produces smoother results')

In [ ]:
# Compare best moving average vs best Gaussian
best_ma = moving_average(noisy, window_size=11)
best_gauss = gaussian_smooth(noisy, sigma=2.5)

# Compute smoothness metric (std of second derivative)
def smoothness_metric(signal):
    second_deriv = np.diff(signal, n=2)
    return np.std(second_deriv)

ma_smoothness = smoothness_metric(best_ma)
gauss_smoothness = smoothness_metric(best_gauss)

# Side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(t, noisy, 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
axes[0].plot(t, clean, 'k--', label='Clean', linewidth=1.5, alpha=0.7)
axes[0].plot(t, best_ma, 'b-', label='Moving Average (w=11)', linewidth=2)
axes[0].set_xlabel('Time (seconds)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title(f'Moving Average\nSmoothness: {ma_smoothness:.4f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(t, noisy, 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
axes[1].plot(t, clean, 'k--', label='Clean', linewidth=1.5, alpha=0.7)
axes[1].plot(t, best_gauss, 'r-', label='Gaussian (σ=2.5)', linewidth=2)
axes[1].set_xlabel('Time (seconds)')
axes[1].set_ylabel('Amplitude')
axes[1].set_title(f'Gaussian Filter\nSmoothness: {gauss_smoothness:.4f}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\nComparison:')
print(f'Moving Average smoothness: {ma_smoothness:.4f}')
print(f'Gaussian smoothness: {gauss_smoothness:.4f}')
print('\nLower smoothness metric = smoother signal')

## Calculus Connections

### Discrete Convolution as Riemann Sum

The discrete convolution sum approximates the continuous convolution integral:

$$\sum_{k} f[k]g[n-k] \Delta t \approx \int f(\tau)g(t-\tau) d\tau$$

This is a Riemann sum approximation where $\Delta t$ is the sampling interval.

### Smoothing Reduces High-Order Derivatives

Let's demonstrate how smoothing reduces the magnitude of derivatives.

In [ ]:
# Compute numerical derivatives
dt = t[1] - t[0]

# First derivative (velocity)
noisy_deriv = np.diff(noisy) / dt
smoothed_deriv = np.diff(best_gauss) / dt

# Second derivative (acceleration)
noisy_deriv2 = np.diff(noisy, n=2) / (dt**2)
smoothed_deriv2 = np.diff(best_gauss, n=2) / (dt**2)

# Plot derivatives
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Original signals
axes[0].plot(t, noisy, 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
axes[0].plot(t, best_gauss, 'r-', label='Smoothed', linewidth=2)
axes[0].set_ylabel('Signal')
axes[0].set_title('Original Signals')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# First derivatives
axes[1].plot(t[:-1], noisy_deriv, 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
axes[1].plot(t[:-1], smoothed_deriv, 'r-', label='Smoothed', linewidth=2)
axes[1].set_ylabel('First Derivative')
axes[1].set_title('First Derivatives (Rate of Change)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Second derivatives
axes[2].plot(t[:-2], noisy_deriv2, 'gray', label='Noisy', linewidth=0.5, alpha=0.5)
axes[2].plot(t[:-2], smoothed_deriv2, 'r-', label='Smoothed', linewidth=2)
axes[2].set_xlabel('Time (seconds)')
axes[2].set_ylabel('Second Derivative')
axes[2].set_title('Second Derivatives (Curvature)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Smoothing dramatically reduces high-frequency components (high derivatives)')
print(f'Noisy 2nd derivative std: {np.std(noisy_deriv2):.2f}')
print(f'Smoothed 2nd derivative std: {np.std(smoothed_deriv2):.2f}')
print(f'Reduction factor: {np.std(noisy_deriv2)/np.std(smoothed_deriv2):.1f}x')

In [ ]:
# Save final comparison plot
plt.figure(figsize=(14, 6))
plt.plot(t, noisy, 'gray', label='Noisy Signal', linewidth=0.5, alpha=0.5)
plt.plot(t, clean, 'k--', label='Clean Signal', linewidth=1.5, alpha=0.7)
plt.plot(t, best_ma, 'b-', label='Moving Average (w=11)', linewidth=2)
plt.plot(t, best_gauss, 'r-', label='Gaussian (σ=2.5)', linewidth=2)
plt.xlabel('Time (seconds)', fontsize=12)
plt.ylabel('Amplitude', fontsize=12)
plt.title('Signal Smoothing Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../plots/results.png', dpi=150, bbox_inches='tight')
plt.show()

print('Plot saved to plots/results.png')

## Conclusion

We've demonstrated:

1. **Moving Average**: Fast O(n) smoothing with simple averaging
2. **Gaussian Filter**: Smooth, continuous filtering with controlled frequency response
3. **Calculus Connection**: Discrete operations approximate continuous integrals and derivatives

### Key Takeaways

- Larger window sizes / sigma values produce more smoothing
- Smoothing reduces high-frequency noise (high derivatives)
- Discrete convolution approximates continuous integral convolution
- Trade-off between noise reduction and signal distortion

### Next Steps

Try the interactive web application to experiment with different parameters in real-time!